In [57]:
import numpy as np
import os
from abc import abstractmethod
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from scipy.special import gammaln  

In [3]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [4]:
def assert_preds_correct(your_preds, sklearn_preds) -> bool:
    return np.abs(your_preds - sklearn_preds).sum() == 0

In [ ]:
def assert_probs_correct(your_probs, sklearn_probs) -> bool:
    return np.abs(your_probs - sklearn_probs).mean() < 1e-3

In [6]:
# Не изменяйте код этого класса!
class NaiveBayes:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.params = dict()
        
    # --- PREDICTION ---
        
    def predict(self, x, return_probs=False):
        """
        x - np.array размерности [N, dim], 
        где N - количество экземпляров данных, 
        dim -размерность одного экземпляра (количество признаков).
        
        Возвращает np.array размерности [N], содержащий номера классов для
        соответствующих экземпляров.
        """
        preds = []
        for sample in x:
            preds.append(
                self.predict_single(sample, return_probs=return_probs)
            )
        
        if return_probs:
            return np.array(preds, dtype='float32')
        
        return np.array(preds, dtype='int32')
    
    # Совет: вниманительно изучите файл подсказок к данной лабораторной
    # и сопоставьте код с описанной математикой байесовского классификатора.
    def predict_single(self, x, return_probs=False) -> int:
        """
        Делает предсказание для одного экземпляра данных.
        
        x - np.array размерности dim.
        
        Возвращает номер класса, которому принадлежит x.
        """
        assert len(x.shape) == 1, f'Expected a vector, but received a tensor of shape={x.shape}'
        marginal_prob = self.compute_marginal_probability(x)  # P(x) - безусловная вероятность появления x
        
        probs = []
        for c in range(self.n_classes):                 # c - номер класса
            prior = self.compute_prior(c)               # P(c) - априорная вероятность (вероятность появления класса)
            likelihood = self.compute_likelihood(x, c)  # P(x|c) - вероятность появления x в предположении, что он принаждлежит c
            
            # Используем теорему Байесса для просчёта условной вероятности P(c|x)
            # P(c|x) = P(c) * P(x|c) / P(x)
            prob = prior * likelihood / marginal_prob
            probs.append(prob)
            
        if return_probs:
            return probs
        
        return np.argmax(probs)
    
    # Вычисляет P(x) - безусловная вероятность появления x.
    @abstractmethod
    def compute_marginal_probability(self, x) -> float:
        pass
    
    # Вычисляет P(c) - априорная вероятность появления класса c.
    @abstractmethod
    def compute_prior(self, c) -> float:
        pass
    
    # Вычисляет P(x|c) - вероятность наблюдения экземпляра x в предположении, что он принаждлежит c.
    @abstractmethod
    def compute_likelihood(self, x, c) -> float:
        pass
    
    # --- FITTING ---
    
    def fit(self, x, y):
        self._estimate_prior(y)
        self._estimate_params(x, y)
    
    @abstractmethod
    def _estimate_prior(self, y):
        pass
    
    @abstractmethod
    def _estimate_params(self, x, y):
        pass

## 1. Наивный классификатор Байеса: гауссово распределение

Напишите недостающий код, создайте и обучите модель. 

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

In [7]:
x_train, y_train, x_test, y_test = load_data('gauss')

In [131]:
# P(C_k|x) = P(x|theta) * P(C_k) / P(x)

class NaiveGauss(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте 
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        return sum(self.compute_prior(c) * self.compute_likelihood(x, c) for c in range(self.n_classes))
        
    
    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        return self.params['prior'][c]

    
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        
        mean, var = self.params['mean'][c], self.params['var'][c]
        return np.prod(1 / np.sqrt(2 * np.pi * var) * np.exp(-((x - mean) ** 2) / (2 * var)))

    
    # --- FITTING ---
    
    def _estimate_prior(self, y):
        # Значения априорных вероятностей сохраните в `params` с ключом 'prior'
        self.params['prior'] = [np.mean(y == i) for i in range(self.n_classes)]
    
    def _estimate_params(self, x, y):
        self.params['mean'] = np.array([x[y == c].mean(axis=0) for c in range(self.n_classes)])
        self.params['var'] = np.array([x[y == c].var(axis=0) for c in range(self.n_classes)])

In [9]:
# Создайте и обучите модель
my_ng_clf = NaiveGauss(n_classes=3)
my_ng_clf.fit(x_train, y_train)
Y = my_ng_clf.predict(x_test)

In [10]:
# Оцените качество модели
accuracy_score(Y, y_test)

1.0

In [11]:
# Сравните вашу модель с аналогом sklearn (GaussianNB)
sklearn_ng_clf = GaussianNB()
sklearn_ng_clf.fit(x_train, y_train)
sklearn_Y = sklearn_ng_clf.predict(x_test)

accuracy_score(sklearn_Y, y_test)

1.0

## 2. Доп. задания (любое на выбор, опционально)

### 2.1  Упрощение наивного классификатора Байеса для гауссова распределения

Уберите из класса NaiveBayes 'лишние' вычисления и удалите код, что соответствует этим вычислениям. Под 'лишним' подразумеваются вещи, что не влияют на итоговое решение о принадлежности классу (значения вероятностей при этом могу стать некорректными, но в данном задании это допустимо).

Напишите в клетке ниже код упрощенного 'классификатора Гаусса' и убедитесь, что его ответы (не значения вероятностей) совпадают с ответами классификатора из задания 1. Для сравнения ответов используйте функцию `assert_preds_correct`.

Указание: работайте в предположении, что классы равновероятны.

Подсказка: упростить необходимо метод `predict_single`.

In [ ]:
# Напишите обновленный код модели здесь
class NaiveGaussSimplified(NaiveGauss):

    # Актуален
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте 
        # методы compute_prior и compute_likelihood.
        return 1
    
    def predict_single(self, x, return_probs=False):
        # Так как априорная вероятность P(x) не зависит от C для всех классов она одинаковая, а это значит, что не влияет на вероятность. 
        # https://en.wikipedia.org/wiki/Naive_Bayes_classifier  
        probs = [self.compute_prior(c) * self.compute_likelihood(x, c) for c in range(self.n_classes)]
        
        return probs if return_probs else np.argmax(probs)

In [15]:
# Создайте и обучите модель
my_ngs_clf = NaiveGaussSimplified(n_classes=3)
my_ngs_clf.fit(x_train, y_train)
ngs_Y = my_ngs_clf.predict(x_test)

In [16]:
# Оцените качество модели
accuracy_score(ngs_Y, y_test)

1.0

In [17]:
# Сравните вашу модель с моделью из задания 1
accuracy_score(Y, y_test)

1.0

In [ ]:
# Объясните в комментариях к этой клетке суть проделанных изменений: почему удаленный код является лишним?

# Так как априорная вероятность P(x) не зависит от C для всех классов она одинаковая, а это значит, что не влияет на вероятность.
# https://en.wikipedia.org/wiki/Naive_Bayes_classifier  

### 2.1  Наивный классификатор Байеса: мультиномиальное распределения

Напишите недостающий код, создайте и обучите модель.

Подсказка: в определении функции правдоподобия много факториалов. Для избежания численного переполнения посчитайте сначала логарифм функции правдоподобия (на бумаге), после примените экспоненту для получения значения вероятности.

Пункты оценки:
1. совпадение предсказанных классов с оными у модели sklearn. Для проверки совпадения используйте функцию `assert_preds_correct`.
2. совпадение значений предсказанных вероятностей принадлежности классами с оными у модели sklearn. Значения вероятностей считаются равными, если функция `assert_probs_correct` возвращает True.

Сложность: математический гений.

In [132]:
x_train, y_train, x_test, y_test = load_data('multinomial')

In [133]:
"""
При желании данный класс можно переписать с нуля. Изменения должны сопровождаться комментариями.
"""
class NaiveMultinomial(NaiveBayes):
    def compute_marginal_probability(self, x) -> float:
        # Для просчёта безусловной вероятности используйте 
        # методы compute_prior и compute_likelihood.
        # Напишите свой код здесь
        return 1
    
    def compute_prior(self, c) -> float:
        assert abs(sum(self.params['prior']) - 1.0) < 1e-3, \
            f"Sum of prior probabilities must be equal to 1, but is {sum(self.params['prior'])}"
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'

        return self.params['prior'][c]
    
    def compute_likelihood(self, x, c) -> float:
        assert c < self.n_classes, f'Class index must be < {self.n_classes}, but received {c}.'
        log_factorial_N = gammaln(np.sum(x) + 1)
        log_factorials_x = np.sum(gammaln(x + 1))
        log_p = np.log(self.params['mean'][c])
        
        log_likelihood = log_factorial_N - log_factorials_x + np.sum(x * log_p)
        return np.exp(log_likelihood)

    
    # --- FITTING ---
    
    def _estimate_prior(self, y):
        # Значения априорных вероятностей сохраните в `params` с ключом 'prior'
        self.params['prior'] = np.array([np.mean(y == i) for i in range(self.n_classes)])

    
    def _estimate_params(self, x, y):
        total_counts = np.array([x[y == c].sum(axis=0) + 1 for c in range(self.n_classes)])
        class_sums = total_counts.sum(axis=1, keepdims=True)
        
        self.params['mean'] = total_counts / class_sums

    # def predict_single(self, x, return_probs=False) -> int:
    #     probs = [self.compute_prior(c) * self.compute_likelihood(x, c) for c in range(self.n_classes)]
        
    #     return probs if return_probs else np.argmax(probs)

In [134]:
# Создайте и обучите модель
nm_clf = NaiveMultinomial(n_classes=2)
nm_clf.fit(x_train, y_train)
nm_Y_preds = nm_clf.predict(x_test, return_probs=False)
nm_Y_probs = nm_clf.predict(x_test, return_probs=True)

In [135]:
# Сравните вашу модель с аналогом sklearn (MultinomialNB)
sklearn_ng_clf = MultinomialNB(alpha=1.0)
sklearn_ng_clf.fit(x_train, y_train)
sklearn_Y_preds = sklearn_ng_clf.predict(x_test)
sklearn_Y_probs = sklearn_ng_clf.predict_proba(x_test)

In [136]:
# Оцените качество модели
print(f"Точность написанной модели: {accuracy_score(nm_Y_preds, y_test)}")
print(f"Точность библиотечной модели: {accuracy_score(sklearn_Y_preds, y_test)}")

Точность написанной модели: 0.8173076923076923
Точность библиотечной модели: 0.8173076923076923


In [ ]:
preds_correct = assert_preds_correct(nm_Y_preds, sklearn_Y_preds)
prods_correct = assert_probs_correct(nm_Y_probs, sklearn_Y_probs)

print(f"Совпадение предсказанных классов: {preds_correct}")
print(f"Совпадение предсказанных вероятностей: {prods_correct}") 
# Различие вероятностей может быть в том, что в библиотечной моделе используется другая формула для мультиномиального классификатора

Совпадение предсказанных классов: True
Совпадение предсказанных вероятностей: False
